In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, adjusted_rand_score
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from scipy.spatial.distance import pdist
import warnings

# Set up plotting
plt.style.use('seaborn-whitegrid')
sns.set(font_scale=1.2)
plt.rcParams['figure.figsize'] = [12, 8]
warnings.filterwarnings('ignore')

# Load the trip features data generated in notebook 02
print("Loading trip features data...")
trip_features_df = pd.read_csv('engineered_trip_features.csv')

print(f"Loaded {len(trip_features_df)} trips with {len(trip_features_df.columns)} features")
print(f"Fishing trips: {trip_features_df['is_fishing'].sum()} ({trip_features_df['is_fishing'].mean()*100:.1f}%)")

Loading trip features data...
Loaded 3591 trips with 52 features
Fishing trips: 2495 (69.5%)


In [4]:
# ##############################################################################
# Section 1: Clustering Validation and Stability Analysis
# ##############################################################################

"""
First, we'll validate the K-means clusters found in notebook 02 and assess
their stability and meaningfulness using established metrics.
"""

print("\nSection 1: Validating cluster quality...")

# Prepare the same feature set used in notebook 02
cluster_features = trip_features_df.drop(['trip_id', 'mmsi', 'start_time', 'end_time', 'is_fishing'], axis=1)

# Handle missing values
for col in cluster_features.columns:
    if cluster_features[col].isna().any():
        cluster_features[col] = cluster_features[col].fillna(cluster_features[col].median())

# Standardize features for clustering
scaler = StandardScaler()
scaled_features = scaler.fit_transform(cluster_features)

print(f"Using {scaled_features.shape[1]} features for clustering analysis")

# Evaluate different numbers of clusters using multiple metrics
k_range = range(2, 11)
metrics = {
    'inertia': [],
    'silhouette': [],
    'calinski_harabasz': []
}

from sklearn.metrics import calinski_harabasz_score

for k in k_range:
    # Fit K-means
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(scaled_features)
    
    # Calculate metrics
    metrics['inertia'].append(kmeans.inertia_)
    metrics['silhouette'].append(silhouette_score(scaled_features, cluster_labels))
    metrics['calinski_harabasz'].append(calinski_harabasz_score(scaled_features, cluster_labels))

# Plot clustering validation metrics
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Elbow curve (inertia)
axes[0].plot(k_range, metrics['inertia'], 'o-')
axes[0].set_title('Elbow Method (Inertia)')
axes[0].set_xlabel('Number of Clusters (k)')
axes[0].set_ylabel('Inertia')
axes[0].grid(True)

# Silhouette score
axes[1].plot(k_range, metrics['silhouette'], 'o-', color='orange')
axes[1].set_title('Silhouette Score')
axes[1].set_xlabel('Number of Clusters (k)')
axes[1].set_ylabel('Silhouette Score')
axes[1].grid(True)

# Calinski-Harabasz score
axes[2].plot(k_range, metrics['calinski_harabasz'], 'o-', color='green')
axes[2].set_title('Calinski-Harabasz Score')
axes[2].set_xlabel('Number of Clusters (k)')
axes[2].set_ylabel('CH Score')
axes[2].grid(True)

plt.tight_layout()
plt.savefig('clustering_validation_metrics.png')
plt.close()

# Determine optimal k based on metrics
optimal_k_silhouette = k_range[np.argmax(metrics['silhouette'])]
optimal_k_ch = k_range[np.argmax(metrics['calinski_harabasz'])]

print(f"\nOptimal number of clusters:")
print(f"  Silhouette score: k = {optimal_k_silhouette} (score: {max(metrics['silhouette']):.3f})")
print(f"  Calinski-Harabasz: k = {optimal_k_ch} (score: {max(metrics['calinski_harabasz']):.1f})")

# Use the optimal k for detailed analysis
optimal_k = optimal_k_silhouette  # Choose based on silhouette score


Section 1: Validating cluster quality...
Using 47 features for clustering analysis

Optimal number of clusters:
  Silhouette score: k = 2 (score: 0.213)
  Calinski-Harabasz: k = 2 (score: 753.1)


In [5]:

# ##############################################################################
# Section 2: Detailed Cluster Analysis
# ##############################################################################

"""
Analyze the optimal clusters in detail to understand what behavioral patterns
they represent and how they relate to fishing activity.
"""

print(f"\nSection 2: Detailed analysis with k = {optimal_k}...")

# Fit final clustering model
final_kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
trip_features_df['cluster'] = final_kmeans.fit_predict(scaled_features)

# Calculate comprehensive cluster statistics
cluster_analysis = []

for cluster_id in range(optimal_k):
    cluster_data = trip_features_df[trip_features_df['cluster'] == cluster_id]
    
    analysis = {
        'cluster': cluster_id,
        'count': len(cluster_data),
        'percentage': len(cluster_data) / len(trip_features_df) * 100,
        
        # Fishing behavior
        'fishing_proportion_mean': cluster_data['fishing_proportion'].mean(),
        'fishing_proportion_std': cluster_data['fishing_proportion'].std(),
        'is_fishing_rate': cluster_data['is_fishing'].mean(),
        
        # Trip characteristics
        'duration_hours_mean': cluster_data['duration_hours'].mean(),
        'duration_hours_std': cluster_data['duration_hours'].std(),
        'point_count_mean': cluster_data['point_count'].mean(),
        
        # Movement patterns
        'speed_mean_avg': cluster_data['speed_mean'].mean(),
        'speed_std_avg': cluster_data['speed_std'].mean(),
        'path_efficiency_avg': cluster_data['path_efficiency'].mean(),
        'course_changes_per_hour_avg': cluster_data['course_changes_per_hour'].mean(),
        
        # Spatial patterns
        'distance_from_shore_mean_avg': cluster_data['distance_from_shore_mean'].mean(),
        'distance_from_port_mean_avg': cluster_data['distance_from_port_mean'].mean(),
    }
    
    cluster_analysis.append(analysis)

cluster_analysis_df = pd.DataFrame(cluster_analysis)

print("\nCluster Analysis Summary:")
print("=" * 80)
for _, cluster in cluster_analysis_df.iterrows():
    print(f"\nCluster {int(cluster['cluster'])} ({cluster['count']} trips, {cluster['percentage']:.1f}%):")
    print(f"  Fishing rate: {cluster['is_fishing_rate']:.1%}")
    print(f"  Avg duration: {cluster['duration_hours_mean']:.1f} hours")
    print(f"  Avg speed: {cluster['speed_mean_avg']:.1f} knots")
    print(f"  Path efficiency: {cluster['path_efficiency_avg']:.3f}")
    print(f"  Course changes/hour: {cluster['course_changes_per_hour_avg']:.2f}")

# Visualize cluster characteristics
key_metrics = [
    'fishing_proportion_mean', 'duration_hours_mean', 'speed_mean_avg', 
    'path_efficiency_avg', 'course_changes_per_hour_avg'
]

fig, axes = plt.subplots(len(key_metrics), 1, figsize=(12, 4*len(key_metrics)))

for i, metric in enumerate(key_metrics):
    sns.barplot(x='cluster', y=metric, data=cluster_analysis_df, ax=axes[i])
    axes[i].set_title(f'{metric.replace("_", " ").title()} by Cluster')
    axes[i].set_xlabel('Cluster')

plt.tight_layout()
plt.savefig('cluster_characteristics_detailed.png')
plt.close()



Section 2: Detailed analysis with k = 2...

Cluster Analysis Summary:

Cluster 0 (2491.0 trips, 69.4%):
  Fishing rate: 67.2%
  Avg duration: 12.9 hours
  Avg speed: 5.6 knots
  Path efficiency: 0.686
  Course changes/hour: 0.81

Cluster 1 (1100.0 trips, 30.6%):
  Fishing rate: 74.6%
  Avg duration: 11.2 hours
  Avg speed: 6.6 knots
  Path efficiency: 0.767
  Course changes/hour: 0.89


In [6]:

# ##############################################################################
# Section 3: Alternative Clustering Methods
# ##############################################################################

"""
Compare K-means results with other clustering algorithms to validate
our findings and potentially discover different patterns.
"""

print("\nSection 3: Comparing clustering methods...")

# 1. DBSCAN Clustering
print("Applying DBSCAN clustering...")

# Try different epsilon values for DBSCAN
eps_values = [0.3, 0.5, 0.7, 1.0]
dbscan_results = []

for eps in eps_values:
    dbscan = DBSCAN(eps=eps, min_samples=10)
    dbscan_labels = dbscan.fit_predict(scaled_features)
    
    n_clusters = len(set(dbscan_labels)) - (1 if -1 in dbscan_labels else 0)
    n_noise = list(dbscan_labels).count(-1)
    
    if n_clusters > 1:
        # Calculate silhouette score (excluding noise points)
        mask = dbscan_labels != -1
        if mask.sum() > 10:  # Need enough points for silhouette score
            sil_score = silhouette_score(scaled_features[mask], dbscan_labels[mask])
        else:
            sil_score = -1
    else:
        sil_score = -1
    
    dbscan_results.append({
        'eps': eps,
        'n_clusters': n_clusters,
        'n_noise': n_noise,
        'silhouette': sil_score
    })

dbscan_df = pd.DataFrame(dbscan_results)
print("\nDBSCAN Results:")
print(dbscan_df)

# Select best DBSCAN result
best_dbscan = dbscan_df[dbscan_df['silhouette'] == dbscan_df['silhouette'].max()]
if not best_dbscan.empty:
    best_eps = best_dbscan.iloc[0]['eps']
    print(f"\nBest DBSCAN: eps={best_eps}, silhouette={best_dbscan.iloc[0]['silhouette']:.3f}")
    
    # Apply best DBSCAN
    dbscan_final = DBSCAN(eps=best_eps, min_samples=10)
    dbscan_labels = dbscan_final.fit_predict(scaled_features)
    trip_features_df['dbscan_cluster'] = dbscan_labels

# 2. Hierarchical Clustering
print("\nApplying Hierarchical clustering...")

# Perform hierarchical clustering
linkage_matrix = linkage(scaled_features[:1000], method='ward')  # Sample for efficiency

# Plot dendrogram
plt.figure(figsize=(12, 8))
dendrogram(linkage_matrix, truncate_mode='level', p=10)
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('Sample Index')
plt.ylabel('Distance')
plt.savefig('hierarchical_dendrogram.png')
plt.close()

# Cut dendrogram to get clusters
hierarchical_labels = fcluster(linkage_matrix, optimal_k, criterion='maxclust')

# Extend to full dataset (simplified approach)
# In practice, you'd apply hierarchical clustering to full dataset
trip_features_df['hierarchical_cluster'] = np.nan
trip_features_df.iloc[:1000, trip_features_df.columns.get_loc('hierarchical_cluster')] = hierarchical_labels



Section 3: Comparing clustering methods...
Applying DBSCAN clustering...

DBSCAN Results:
   eps  n_clusters  n_noise  silhouette
0  0.3           0     3591          -1
1  0.5           0     3591          -1
2  0.7           0     3591          -1
3  1.0           0     3591          -1

Best DBSCAN: eps=0.3, silhouette=-1.000

Applying Hierarchical clustering...


In [7]:

# ##############################################################################
# Section 4: Cluster Interpretation and Business Insights
# ##############################################################################

"""
Interpret the clusters in terms of fishing operations and validate that
they correspond to meaningful behavioral patterns.
"""

print("\nSection 4: Business interpretation of clusters...")

# Define cluster interpretations based on characteristics
def interpret_cluster(cluster_data):
    """Interpret cluster based on its characteristics."""
    fishing_rate = cluster_data['is_fishing_rate']
    avg_speed = cluster_data['speed_mean_avg']
    duration = cluster_data['duration_hours_mean']
    efficiency = cluster_data['path_efficiency_avg']
    
    if fishing_rate > 0.8 and avg_speed < 5 and efficiency < 0.7:
        return "Active Fishing Operations"
    elif fishing_rate > 0.7 and avg_speed < 6 and efficiency < 0.8:
        return "Mixed Fishing/Transit"
    elif fishing_rate < 0.4 and avg_speed > 7 and efficiency > 0.9:
        return "Pure Transit/Travel"
    elif fishing_rate > 0.6 and avg_speed < 4:
        return "Intensive Fishing (Slow)"
    else:
        return "Mixed Operations"

# Apply interpretations
cluster_analysis_df['interpretation'] = cluster_analysis_df.apply(interpret_cluster, axis=1)

print("\nCluster Interpretations:")
print("=" * 50)
for _, cluster in cluster_analysis_df.iterrows():
    print(f"Cluster {int(cluster['cluster'])}: {cluster['interpretation']}")

# Validate interpretations with statistical tests
from scipy.stats import f_oneway

print("\nStatistical validation of cluster differences:")

# Test if clusters are significantly different in key metrics
metrics_to_test = ['speed_mean', 'fishing_proportion', 'path_efficiency', 'duration_hours']

for metric in metrics_to_test:
    cluster_groups = [trip_features_df[trip_features_df['cluster'] == i][metric].values 
                     for i in range(optimal_k)]
    
    # Remove empty groups
    cluster_groups = [group for group in cluster_groups if len(group) > 0]
    
    if len(cluster_groups) > 1:
        f_stat, p_value = f_oneway(*cluster_groups)
        print(f"  {metric}: F={f_stat:.2f}, p={p_value:.6f} {'***' if p_value < 0.001 else '**' if p_value < 0.01 else '*' if p_value < 0.05 else ''}")



Section 4: Business interpretation of clusters...

Cluster Interpretations:
Cluster 0: Mixed Operations
Cluster 1: Mixed Operations

Statistical validation of cluster differences:
  speed_mean: F=111.00, p=0.000000 ***
  fishing_proportion: F=21.96, p=0.000003 ***
  path_efficiency: F=44.42, p=0.000000 ***
  duration_hours: F=5.21, p=0.022532 *


In [8]:

# ##############################################################################
# Section 5: Clustering Insights for Supervised Learning
# ##############################################################################

"""
Investigate whether cluster information can improve our supervised
fishing detection model.
"""

print("\nSection 5: Using clusters to improve supervised learning...")

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report

# Prepare data for supervised learning with cluster features
vessels = trip_features_df['mmsi'].unique()
train_vessels, test_vessels = train_test_split(vessels, test_size=0.2, random_state=42)

# Features for modeling
model_features = [col for col in trip_features_df.columns 
                 if col not in ['trip_id', 'mmsi', 'start_time', 'end_time', 'is_fishing', 'fishing_proportion']]

# Test 1: Original features only (baseline)
baseline_features = [f for f in model_features if 'cluster' not in f]
X_baseline = trip_features_df[baseline_features]
y = trip_features_df['is_fishing']

# Handle missing values
for col in X_baseline.columns:
    if X_baseline[col].isna().any():
        X_baseline[col] = X_baseline[col].fillna(X_baseline[col].median())

X_train_baseline = X_baseline[trip_features_df['mmsi'].isin(train_vessels)]
X_test_baseline = X_baseline[trip_features_df['mmsi'].isin(test_vessels)]
y_train = y[trip_features_df['mmsi'].isin(train_vessels)]
y_test = y[trip_features_df['mmsi'].isin(test_vessels)]

# Train baseline model
baseline_model = RandomForestClassifier(n_estimators=100, random_state=42)
baseline_model.fit(X_train_baseline, y_train)
baseline_score = baseline_model.score(X_test_baseline, y_test)

print(f"Baseline model accuracy: {baseline_score:.4f}")

# Test 2: Features + cluster information
cluster_features_list = ['cluster']  # Add other cluster variants if available
if 'dbscan_cluster' in trip_features_df.columns:
    cluster_features_list.append('dbscan_cluster')

enhanced_features = baseline_features + cluster_features_list
X_enhanced = trip_features_df[enhanced_features]

for cluster_col in cluster_features_list:
    if cluster_col in X_enhanced.columns:
        cluster_dummies = pd.get_dummies(X_enhanced[cluster_col], prefix=cluster_col)
        X_enhanced = pd.concat([X_enhanced.drop(cluster_col, axis=1), cluster_dummies], axis=1)

# Handle missing values
for col in X_enhanced.columns:
    if X_enhanced[col].isna().any():
        X_enhanced[col] = X_enhanced[col].fillna(X_enhanced[col].median())

X_train_enhanced = X_enhanced[trip_features_df['mmsi'].isin(train_vessels)]
X_test_enhanced = X_enhanced[trip_features_df['mmsi'].isin(test_vessels)]

# Train enhanced model
enhanced_model = RandomForestClassifier(n_estimators=100, random_state=42)
enhanced_model.fit(X_train_enhanced, y_train)
enhanced_score = enhanced_model.score(X_test_enhanced, y_test)

print(f"Enhanced model accuracy: {enhanced_score:.4f}")
print(f"Improvement: {enhanced_score - baseline_score:.4f}")

# Test statistical significance of improvement
if enhanced_score > baseline_score:
    print("Cluster information provides improvement to supervised learning")
else:
    print("Cluster information does not significantly improve supervised learning")



Section 5: Using clusters to improve supervised learning...
Baseline model accuracy: 0.8663
Enhanced model accuracy: 0.8552
Improvement: -0.0111
Cluster information does not significantly improve supervised learning
